In [3]:
import os
from dotenv import load_dotenv 


In [4]:
load_dotenv()

import os
os.environ["OPENAI_API_KEY"]= 

In [5]:
from langchain_openai.chat_models import ChatOpenAI



In [185]:
#import Lbraries
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
import os
from langchain_core.messages import SystemMessage
from langchain.memory import ConversationBufferWindowMemory
from langchain.tools.retriever import create_retriever_tool
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_experimental.text_splitter import SemanticChunker
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [7]:
load_dotenv()

True

In [8]:
openai_api_key = os.getenv("OPENAI_API_KEY")


In [10]:

os.getenv("PINECONE_API_KEY")

'93072522-ca0d-440f-b702-dfecafedce05'

Load file from directory


In [11]:
def load_file(data):
    loader = DirectoryLoader("data",
                    glob="**/*.txt",
                    loader_cls= TextLoader)
    text_documents = loader.load()
    return text_documents
     


In [147]:
#extract the data 
extracted_data = load_file([ "2024-03-12_Dev_Meeting_-_Knowledge_Now_summary_1.txt", "2024-03-12-Dev-Meeting-Knowledge-Now-Transcrpt-2.txt"])
print(extracted_data)

[Document(page_content="\n01:09\nAlejo\nOkay. Hi, see you all again. \n\n01:17\nBartek\nWaiting for us or it's only three of us here? Hi guys, I'm hearing you right now. I just had a dinner, so excuse me, eating. Aleo, I think you have something with your audio. I can't hear you. Aleo, you're not here. \n\n02:49\nAlejo\nOkay, I guess no headphones it is. Hey you guys. \n\n02:53\nBartek\nYeah, sounds good. \n\n02:54\nAlejo\nCool. \n\n02:55\nBartek\nHi again. \n\n02:57\nAlejo\nHey, did you say something? Is there something you wanted to share? \n\n03:05\nBartek\nI just wanted to share that I have great. Dinner or lunch? Lunch. I messed that up every time. Lunch. \n\n03:13\nKuba\nHi guys. \n\n03:15\nAlejo\nHey Kuba. \n\n03:23\nKuba\nLet'S see how you doing, bro? \n\n03:35\nAlejo\n1 second, you guys, thank you guys for sharing your update on Friday. It makes me very happy to see progress on what you get to work on or what you want to work on. I hear a lot of rag, I hear a lot of lang chain

In [84]:
#split documents into chunks
def text_split(extracted_data):
    text_splitter = SemanticChunker(OpenAIEmbeddings(), breakpoint_threshold_type="percentile")
    documents = text_splitter.split_documents(extracted_data)

    return documents


In [85]:
text_chunk = text_split(extracted_data)
print("The length of the chunk:", len(text_chunk))

The length of the chunk: 130


In [88]:
print(text_chunk[1].page_content)

Lunch. I messed that up every time. Lunch. 03:13
Kuba
Hi guys. 03:15
Alejo
Hey Kuba. 03:23
Kuba
Let'S see how you doing, bro? 03:35
Alejo
1 second, you guys, thank you guys for sharing your update on Friday. It makes me very happy to see progress on what you get to work on or what you want to work on. I hear a lot of rag, I hear a lot of lang chain. As you said, kuwa, rag is the future. And it's been very exciting to talk to developers this last week. A variety of experiences and expertise and they all say the same thing. It's like Rag is where it's at. Rag is the future, Rag is what I want to learn. So we're taking that strong indication and running with it. We're building knowledge now. Okay, first of all, before we step into that, because start rattling on, wanted to share an update. I'm working on the real estate automations. 04:57
Alejo
I'm working on the free side on the two free real estate automations. When those are ready, I'll share the demo. In the next few days. They'll be 

In [89]:
model_name = "text-embedding-3-large"

def data_embedding():
    embeddings = OpenAIEmbeddings()
    return embeddings



In [90]:
embeddings = data_embedding()

In [91]:
import pinecone

In [92]:
YOUR_API_KEY = os.getenv("PINECONE_API_KEY")

Your_env = "gcp-starter"

In [93]:
from langchain_pinecone import PineconeVectorStore

index_name = "knowledgenow"

In [94]:
#Adding embeddings into the vectorstore

vectorstore = PineconeVectorStore.from_documents(text_chunk, embeddings, index_name=index_name)

In [135]:
#initializing retrival
retriver = vectorstore.as_retriever(search_kwargs={'k':25})

In [186]:
#initiaze language models
llm  = ChatOpenAI(
    openai_api_key=openai_api_key, 
    model="gpt-4-turbo-preview", 
    temperature= 0.0,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]

    )

In [148]:
from langchain. retrievers.multi_query import MultiQueryRetriever

retriever = MultiQueryRetriever.from_llm(
    retriever=retriver, llm=llm
)

In [154]:
import logging 

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [156]:
query = "what did alejo and bartek discuss in the meeting"

docs = retriever.get_relevant_documents(query=query)

len(docs)


INFO:langchain.retrievers.multi_query:Generated queries: ['1. What were the topics of conversation between Alejo and Bartek during their meeting?', '2. Can you detail the subjects Alejo and Bartek covered in their recent meeting?', '3. What did Alejo and Bartek talk about in their conference?']


28

In [179]:
print(docs)

[Document(page_content="\n01:09\nAlejo\nOkay. Hi, see you all again. 01:17\nBartek\nWaiting for us or it's only three of us here? Hi guys, I'm hearing you right now. I just had a dinner, so excuse me, eating. Aleo, I think you have something with your audio. I can't hear you. Aleo, you're not here. 02:49\nAlejo\nOkay, I guess no headphones it is. Hey you guys. 02:53\nBartek\nYeah, sounds good. 02:54\nAlejo\nCool. 02:55\nBartek\nHi again. 02:57\nAlejo\nHey, did you say something? Is there something you wanted to share? 03:05\nBartek\nI just wanted to share that I have great. Dinner or lunch?", metadata={'source': 'data\\2024-03-12-Dev-Meeting-Knowledge-Now-Transcrpt-2.txt'}), Document(page_content="Happy to work on this one. Seems that it's going to be revolutionizing. 02:08:56\nAlejo\nYeah, absolutely. We can help a lot of people make better lifestyles, better work, better everything. 02:09:04\nBartek\nYeah. 02:09:04\nAlejo\nTake care, brother. I'll see you tomorrow. 02:09:07\nBartek\n

In [160]:
#conversational memory
conversational_memory = ConversationBufferWindowMemory(
    memory_key= "chat_history",
    k=5,
    return_messages=True
)

In [162]:
#retrival chain
qa = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever
)


In [100]:
from langchain.tools import tool

@tool
def Knowledgebase(query:str) -> str:
    """Use this tool when answering questions about meetings, task, or general about a company called fastrise"""
    result_str = retriver
    return result_str

In [164]:
#creating a custom tool
@tool
def Knowledgebase(query:str)-> str:
    """
    Use this tool when answering question about meeting, task, activities in a organization called Fastrise"""
    result_str = qa.invoke(query)
    return result_str



In [165]:
global_context = """Fasterise

### Mission and Vision
- *Mission and Vision*: Fasterise is centered around leveraging AI technology to amplify human potential, prioritizing ethical AI development that enriches human capabilities. The company envisions a future where AI solutions address societal needs and business challenges while augmenting the human potential and keeping it at the center of the system.
### Team Structure and Roles
- *Henry*: Brings AI and entrepreneurial insights, contributing significantly to the team's innovative capabilities in AI and LangChain.
- *Zach*: Offers sales expertise and a keen sense for impactful business development, essential for guiding the company's market direction.
- *Paige*: With a background in education and a focus on ethical AI, Paige drives the mission, marketing vision, and operational strategies of the company.
- *Pawel*: Provides business analysis and client-facing skills, ensuring professional and confident representation of Fasterise's image.
- *Bartek*: Leads in engineering with expertise in Prompt Engineering and AI Agents, and a focus on system design and test-driven development, enhancing the company's technical rigor.
- *Alejo*: Plays a key role in Operations, Sales, and Development. In development, Alejo focuses on developer guidance through Scrums, making the vision of the company more tangible into projects with a particular emphasis on AI technologies like RAG, LLMs and Multi-Agent Systems.
- *Hubi*: is diving into Python, LangChain and RAG. He has experience in front end development.
- *Jorg*: Contributes to social media content and marketing campaigns, showcasing the company's solutions through various platforms.
- *Kuba*: has played a key role in LangChain and RAG efforts, creating the first versions of the Pinecone Vector Database chat and the Personalized Outreach System.
### Projects and Operations
- *Completed Projects*: Highlight projects like the Ski Chalet Concierge, Scheduler Assistant, and Lead Qualifier, which exemplify Fasterise's commitment to enhancing operational efficiency and customer experience through AI.
- *Current Projects*: Emphasize the development of innovative solutions like the Zoom Transcriber to Pinecone Integration, Knowledge Base Builder called KnowledgeNow, Personalized Outreach System, Administrative Automation Tools, and Real Estate-focused AI Solutions. These projects aim to integrate AI across various industries, improving efficiency and amplifying human capabilities.
- *Future Direction*: Focus on continuous exploration of AI applications, with a commitment to adapting to technological advancements and exploring areas where AI can have a transformative societal impact, keeping empowerement and education at the forefront of the company's mission.
### Company Culture and Values
- *Profit-Sharing and Collaboration*: All team members are co-founders, with a profit-sharing structure and collective decision-making. The company culture promotes innovation, ethical business practices, and a shared vision for the impact of AI technology.
- *Ethical and Human-Centered Approach*: A foundational commitment to developing AI solutions that align with ethical standards, enhance rather than replace human work, and focus on amplifying human capabilities.
### Innovations and Products
- *Technological Innovations*: AI-driven products, including AI solutions for real estate, personalized outreach, and KnowledgeNow. High technological sophistication and human-centered design of products like automated CRM and calendar systems, which facilitate better interactions with leads and clients.
### Conclusion
Fasterise stands out for its innovative approach to AI technology, with a strong emphasis on ethical development, collaborative innovation, and a culture of shared success. Through a diverse range of projects, the company showcases its capability to address complex challenges across various sectors, aiming for a significant societal impact and leading by example in the ethical use and development of AI.
"""

In [166]:
#creating a prompt template

template = """
 Your role is to use the global context to answer any question from users. 


 {chat_history}

 global_context: {global_context}

Begin!

 Question: {input}

 Thoughts: {agent_scratchpad}
 
 """

prompt = PromptTemplate.from_template(template=template, MessagePlaceholder=["(agent_scratchpad)"])



In [167]:
tools = [Knowledgebase]

In [168]:
from langchain.agents import create_openai_functions_agent

agent = create_openai_functions_agent(llm=llm, tools=tools, prompt=prompt)

In [169]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [170]:
query ="what project was bartek and Alejo discuss in the meeting"

out = agent_executor.invoke({"input": query,
                       "chat_history": [],
                       "global_context":global_context })

print(out["output"])






> Entering new AgentExecutor chain...

Invoking: `Knowledgebase` with `{'query': 'what project was bartek and Alejo discuss in the meeting'}`




INFO:langchain.retrievers.multi_query:Generated queries: ['1. What was the topic of the project discussed by Bartek and Alejo during the meeting?', '2. Which project were Bartek and Alejo talking about in their recent meeting?', '3. What subject matter were Bartek and Alejo focusing on in their project discussion at the meeting?']


{'question': 'what project was bartek and Alejo discuss in the meeting', 'answer': 'Bartek and Alejo discussed the project "Knowledgenow" during the meeting. This project involves creating a system that generates a list of tasks for team members after each meeting to provide clarity on the tasks assigned to each individual. The system includes various agents such as context enrichment, rag extractor, evaluator, QA, supervisor, merger, and answer formatting agents. The project aims to automate tasks like cleaning and rendering transcripts for system input and includes use cases like messaging bots providing assignment updates post-meetings. The team is working towards achieving Minimum Viable Product (MVP) status and plans to include features like contribution calculators or project log automation in future iterations.\n\n', 'sources': 'data\\2024-03-12-Dev-Meeting-Knowledge-Now-Transcrpt-2.txt, data\\2024-03-12_Dev_Meeting_-_Knowledge_Now_summary_1.txt'}Bartek and Alejo discussed the p

In [171]:


from langchain.memory import ConversationBufferMemory

#conversational memory 
conversational_memory = ConversationBufferWindowMemory(
    memory_key="chat_history",
    k=5,
    return_messages=True

)

In [172]:
conversational_memory.chat_memory.messages

[]

In [173]:
conversational_memory.chat_memory.add_user_message(query)
conversational_memory.chat_memory.add_ai_message(out["output"])

In [174]:
conversational_memory.chat_memory.messages

[HumanMessage(content='what project was bartek and Alejo discuss in the meeting'),
 AIMessage(content='Bartek and Alejo discussed the project "Knowledgenow" during the meeting. This project involves creating a system that generates a list of tasks for team members after each meeting to provide clarity on the tasks assigned to each individual. The system includes various agents such as context enrichment, rag extractor, evaluator, QA, supervisor, merger, and answer formatting agents. The project aims to automate tasks like cleaning and rendering transcripts for system input and includes use cases like messaging bots providing assignment updates post-meetings. The team is working towards achieving Minimum Viable Product (MVP) status and plans to include features like contribution calculators or project log automation in future iterations.')]

In [175]:
from langchain_core.messages.human import HumanMessage

def memory2str(memory:ConversationBufferWindowMemory):
    messages = memory.chat_memory.messages
    memory_list = [
        f"Human: {mem.content}" for mem in messages 
    ]
    memory_str = "/n".join(memory_list)
    return memory_str

In [176]:
print(memory2str(conversational_memory))

Human: what project was bartek and Alejo discuss in the meeting/nHuman: Bartek and Alejo discussed the project "Knowledgenow" during the meeting. This project involves creating a system that generates a list of tasks for team members after each meeting to provide clarity on the tasks assigned to each individual. The system includes various agents such as context enrichment, rag extractor, evaluator, QA, supervisor, merger, and answer formatting agents. The project aims to automate tasks like cleaning and rendering transcripts for system input and includes use cases like messaging bots providing assignment updates post-meetings. The team is working towards achieving Minimum Viable Product (MVP) status and plans to include features like contribution calculators or project log automation in future iterations.


In [177]:
def chat(text:str):
    out = agent_executor.invoke({
        "input":text, 
        "chat_history": [memory2str(conversational_memory)], 
        "global_context": global_context})
    conversational_memory.chat_memory.add_user_message(text)
    conversational_memory.chat_memory.add_ai_message(out["output"])
    return out["output"]

In [178]:
print(chat("list out the tasks alejo and bartek were talking about in the meeting"))



> Entering new AgentExecutor chain...
In the meeting, Bartek and Alejo discussed the project "Knowledgenow," which involves creating a system that generates a list of tasks for team members after each meeting. The tasks they likely talked about include:

1. **Context Enrichment**: Developing a system to enhance the context of the meeting discussions, making it easier for the system to understand and categorize the content of the meetings.

2. **RAG Extractor**: Working on a Retrieval-Augmented Generation (RAG) extractor to pull relevant information from various sources to support the generation of tasks.

3. **Evaluator**: Creating an evaluator agent to assess the relevance and accuracy of the generated tasks, ensuring they align with the meeting's outcomes.

4. **QA (Quality Assurance)**: Implementing a QA system to check the quality of the tasks generated, ensuring they meet the project's standards.

5. **Supervisor**: Designing a supervisor agent to oversee the entire process of t

In [118]:
print(chat("can you list out the tasks alejo and bartek were talking about in the meeting"))



> Entering new AgentExecutor chain...
In the meeting, Alejo and Bartek discussed working on a project that they believe will be revolutionary, aiming to help a lot of people improve their lifestyles, work, and overall well-being. While the specific tasks were not detailed, given their roles and expertise within Fasterise, it's likely that their discussion involved:

- **Development and Implementation of AI Technologies**: Considering Bartek's expertise in engineering, particularly in Prompt Engineering and AI Agents, and Alejo's role in Operations, Sales, and Development with a focus on developer guidance through Scrums, their tasks might involve strategizing and implementing AI technologies like RAG, LLMs, and Multi-Agent Systems to make the company's vision more tangible in projects.

- **Project Management and Scrum Guidance**: Alejo's focus on providing developer guidance through Scrums suggests that part of their discussion could have involved planning sprints, setting project m

In [ ]:
while True:
    user_input=input(f"Input_Prompt:")
    result= qa({"query": user_input})
    print("Response:", result["result"])

c:\Windows\System32\Knowledgenow\env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Response: It seems like you haven't asked a question yet. How can I assist you today?
